In [4]:
import os
import os.path as osp

import time
import os, mmap
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.utils import add_self_loops
from sklearn.manifold import TSNE
import numpy as np
from torch_geometric.datasets import Planetoid
import nvsmi
import threading
import psutil

# Using MMAP LoadData

In [7]:
from ctypes import *
result = cdll.LoadLibrary("./mainA3.dll")


# str_test="need_bytes"  
# val=str_test.encode()  
# func.strTest.restype = c_char_p
# file = c_char("/mnt/mem/cora.cites")
start_time = time.perf_counter()

file = bytes("/mnt/mem/cora.cites",encoding='utf-8')
result.readMMap.restype = c_char_p
cites = result.readMMap(file, 0, 10000);

file = bytes("/mnt/mem/cora.content",encoding='utf-8')
result.readMMap.restype = c_char_p
content = result.readMMap(file, 0, 200000000);

end_time = time.perf_counter()
# result.closemmap(bytes(m,encoding='utf-8'))

OSError: [WinError 193] %1 不是有效的 Win32 应用程序。

In [3]:
print('Loading time: ' + str(end_time - start_time))

Loading time: 1.0166201250000029


In [46]:
# print(cites.decode())
# print(content.decode())

In [4]:
# 索引字典，将原本的论文id转换到从0开始编码
index_dict = dict()
# 标签字典，将字符串标签转化为数值
label_to_index = dict()

features = []
labels = []
edge_index = []
# start timer
start_time = time.perf_counter()

nodes = content.decode().split("\n")[:-1]
for node in nodes:
    node_info = node.split()
    print(node_info)
    index_dict[int(node_info[0])] = len(index_dict)
    features.append([int(i) for i in node_info[1:-1]])

    label_str = node_info[-1]
    if (label_str not in label_to_index.keys()):
        label_to_index[label_str] = len(label_to_index)
    labels.append(label_to_index[label_str])

edges = cites.decode().split("\n")[:-1]
for edge in edges:
    start, end = edge.split()
    # 训练时将边视为无向的，但原本的边是有向的，因此需要正反添加两次
    edge_index.append([index_dict[int(start)], index_dict[int(end)]])
    edge_index.append([index_dict[int(end)], index_dict[int(start)]])

# end timer
end_time = time.perf_counter()

NameError: name 'content' is not defined

In [48]:
print('Data proccessing time: ' + str(end_time - start_time))

Data proccessing time: 4.886617687996477


# Using DirectIO LoadData

In [2]:
import directio
path = "/mnt/mem/"
cites = path + "cora.cites"
content = path + "cora.content"

# 索引字典，将原本的论文id转换到从0开始编码
index_dict = dict()
# 标签字典，将字符串标签转化为数值
label_to_index = dict()

features = []
labels = []
edge_index = []
# start timer
start_time = time.perf_counter()

# fd = open(cites, "r")
fd_o = os.open(content, os.O_DIRECT | os.O_RDONLY)
# print(fd)
# m = mmap.mmap(fd, 0)
content = directio.read(fd_o,10521600)

fd_o1 = os.open(cites, os.O_DIRECT | os.O_RDONLY)
cites = directio.read(fd_o1,70144)

end_time = time.perf_counter()
print('Loading time: ' + str(end_time - start_time))



Loading time: 0.024830405998727656


In [50]:
start_time = time.perf_counter()
nodes = content.decode().split("\n")[:-1]
for node in nodes:
    node_info = node.split()
    print(node_info)
    index_dict[int(node_info[0])] = len(index_dict)
    features.append([int(i) for i in node_info[1:-1]])

    label_str = node_info[-1]
    if (label_str not in label_to_index.keys()):
        label_to_index[label_str] = len(label_to_index)
    labels.append(label_to_index[label_str])

edges = cites.decode().split("\n")[:-1]
for edge in edges:
    start, end = edge.split()
    # 训练时将边视为无向的，但原本的边是有向的，因此需要正反添加两次
    edge_index.append([index_dict[int(start)], index_dict[int(end)]])
    edge_index.append([index_dict[int(end)], index_dict[int(start)]])
end_time = time.perf_counter()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['235670', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




['46547', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',

In [35]:
print('Data proccessing time: ' + str(end_time - start_time))

Data proccessing time: 1.288203905998671


In [36]:
# To Tensor
labels = torch.LongTensor(labels)
features = torch.FloatTensor(features)
# norm
# features = torch.nn.functional.normalize(features, p=1, dim=1)
edge_index = torch.LongTensor(edge_index)


In [5]:
class GCNNet(torch.nn.Module):
    def __init__(self, num_feature, num_label):
        super(GCNNet, self).__init__()
        self.GCN1 = GCNConv(num_feature, 16)
        self.GCN2 = GCNConv(16, num_label)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.GCN1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.GCN2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [6]:
class GATNet(torch.nn.Module):
    def __init__(self, num_feature, num_label):
        super(GATNet, self).__init__()
        self.GAT1 = GATConv(num_feature, 8, heads=8, concat=True, dropout=0.6)
        self.GAT2 = GATConv(8 * 8, num_label, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.GAT1(x, edge_index)
        x = F.relu(x)
        x = self.GAT2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [7]:
class SAGENet(torch.nn.Module):
    def __init__(self, num_feature, num_label):
        super(SAGENet, self).__init__()
        self.conv1 = SAGEConv(num_feature, 128)
        self.conv2 = SAGEConv(128, num_label)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.softmax(x, dim=1)

        return x

In [9]:
seed = 1234
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)  # Numpy module.
# random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

mask = torch.randperm(len(index_dict))
train_mask = mask[:140]
val_mask = mask[140:640]
test_mask = mask[1708:2708]

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

data = Data(x=features, edge_index=edge_index.t().contiguous(), y=labels).to(device)
print(data)
#model
model = GCNNet(features.shape[1], len(label_to_index)).to(device)

print(model)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


# for epoch in range(1):
#     optimizer.zero_grad()
#     out = model(cora)
#     loss = F.nll_loss(out[train_mask], cora.y[train_mask])
#     #print('epoch: %d loss: %.4f' % (epoch, loss))
#     loss.backward()
#     optimizer.step()

#     if ((epoch + 1) % 10 == 0):
#         model.eval()
#         _, pred = model(cora).max(dim=1)
#         correct = int(pred[test_mask].eq(cora.y[test_mask]).sum().item())
#         acc = correct / len(test_mask)
#         print('Accuracy: {:.4f}'.format(acc))
#         model.train()
        
model.train()
for epoch in range(1000):
    out = model(data)
    loss = criterion(out[train_mask], data.y[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, pred = torch.max(out[train_mask], dim=1)
    correct = (pred == data.y[train_mask]).sum().item()
    acc = correct/len(train_mask)
    print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(
        epoch, loss.item(), acc))
    
    if ((epoch + 1) % 10 == 0):
        model.eval()
        _, pred = model(data).max(dim=1)
        correct = int(pred[test_mask].eq(data.y[test_mask]).sum().item())
        acc = correct / len(test_mask)
        print('Accuracy: {:.4f}'.format(acc))
        model.train()
    
# for epoch in range(200):
#     out = model(data)
#     loss = criterion(out[data.train_mask], data.y[data.train_mask])
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     _, pred = torch.max(out[data.train_mask], dim=1)
#     correct = (pred == data.y[data.train_mask]).sum().item()
#     acc = correct/data.train_mask.sum().item()

#     print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(
#         epoch, loss.item(), acc))

AttributeError: 'list' object has no attribute 't'

In [42]:
out = model(data)
loss = criterion(out[test_mask], data.y[test_mask])
_, pred = torch.max(out[test_mask], dim=1)
correct = (pred == data.y[test_mask]).sum().item()
acc = correct/len(test_mask)
print("test_loss: {:.4f} test_acc: {:.4f}".format(loss.item(), acc))

test_loss: 0.8173 test_acc: 0.7940


# GNN Model

In [4]:
from torch_geometric.datasets import Planetoid
import torch

In [5]:
dataset_cora = Planetoid(root='./cora/', name='Cora')
# dataset = Planetoid(root='./citeseer',name='Citeseer')
# dataset = Planetoid(root='./pubmed/',name='Pubmed')
print(dataset_cora)

Cora()


In [6]:
dataset = dataset_cora

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GATConv

In [29]:
# gpu_data = list(nvsmi.get_gpus())[0]
# start_gpu_util = gpu_data.gpu_util
# start_gpu_mem_use = gpu_data.mem_used
# total_gpu_mem = gpu_data.mem_total
disk_io_counter = psutil.disk_io_counters()
disk_total = disk_io_counter[2] + disk_io_counter[3]  # read_bytes + write_bytes
p = psutil.Process()
group_mem_rss = []
# group_gpu_util = []
# group_gpu_mem_use = []
group_disk_usage = []
group_iowait = []
t_status = True

def get_gpu_info():
#     gpu_data = list(nvsmi.get_gpus())[0]
    while True:
        if not t_status:
            break
#         group_gpu_util.append(gpu_data.gpu_util)
#         group_gpu_mem_use.append(gpu_data.mem_used)
        group_mem_rss.append(psutil.Process(os.getpid()).memory_info().rss)
        io_counters = p.io_counters()
        disk_usage = io_counters[2] + io_counters[3]
        group_disk_usage.append(disk_usage)  # read_bytes + write_bytes
        try:
            a = psutil.cpu_times_percent().iowait
        except:
            a = 0
        group_iowait.append(a)
        print(f"{disk_usage / 1024 / 1024} Mb/s")
        time.sleep(0.09)

In [30]:
path = "/mnt/NVme/project_moka/pubmed/"
path = "/mnt/ramfs/project_moka/pubmed/"
#path = "/mnt/ext4ramdisk/project_moka/pubmed/"
#path = "./pubmed/"

path_Cora = "/mnt/NVme/project_moka/data/Cora/"
# path_Cora = "/mnt/ramfs/project_moka/data/Cora/"
#path_Cora = "/mnt/ext4ramdisk/project_moka/data/Cora/"
#path_Cora = "./data/Cora/"

path_pm = "/mnt/NVme/datasets/"
path_ram = "/mnt/ramfs/datasets/"

times = 15
total_time = 0
total_run_time = 0
batch_size = 128
epoch_num = 20

# pre-load Planetoid
t = threading.Thread(target=get_gpu_info)
t.start()

dataset_test = Planetoid(root="./cora/", name='Cora')

t_status = False
t.join()
# print(f"GPU 显存占用: {np.mean(group_gpu_mem_use)}Mb")
# print(f"GPU 显存占用率: {np.mean(group_gpu_mem_use) * 100 / total_gpu_mem}%")
# print(f"GPU 平均使用率: {np.mean(group_gpu_util) - start_gpu_util}%")
# tmp_ = sum(np.where(np.array(group_gpu_util) - start_gpu_util > 1, True, False))
# print(f"GPU 空闲率: {(len(group_gpu_util) - tmp_) * 100 / len(group_gpu_util)}%")
print(f'内存使用：{np.mean(group_mem_rss) / 1024 / 1024 / 1024:.4f} GB')
print(f'磁盘IO使用：{np.mean(group_disk_usage) / 1024 / 1024 / 1024:.4f} GB/s')
print(f'磁盘IO使用率：{np.mean(group_disk_usage) * 100 / disk_total:.4f}%')
print(f'cpu iowait：{np.mean(group_iowait)}')
#dataset_test = Planetoid(root=path_Cora, name='Cora')
# the dataset for test is shown below (different from the above)
# dataset_test = Planetoid(root='./data/Cora/', name='Cora')


dataset = dataset_test


data = dataset[0]  # Get the first graph object.
from torch_geometric.data import ClusterData, ClusterLoader, DataLoader

torch.manual_seed(1234)
cluster_data = ClusterData(data, num_parts=128)  # 1. Create subgraphs.
train_loader = ClusterLoader(cluster_data, batch_size=batch_size,
                             shuffle=True)  # 2. Stochastic partitioning scheme.

289.03955078125 Mb/s
内存使用：0.4403 GB
磁盘IO使用：0.2823 GB/s
磁盘IO使用率：12.2232%
cpu iowait：0.1


Computing METIS partitioning...
Done!


# Build GNN Network

In [13]:
class SAGENet(nn.Module):
    def __init__(self):
        super(SAGENet, self).__init__()
        self.conv1 = SAGEConv(dataset.num_node_features, 128)
        self.conv2 = SAGEConv(128, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.softmax(x, dim=1)

        return x

In [14]:
class GATNet(torch.nn.Module):
    def __init__(self):
        super(GATNet, self).__init__()
        self.gat1 = GATConv(dataset.num_node_features, 8, 8, dropout=0.6)
        self.gat2 = GATConv(64, 7, 1, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gat1(x, edge_index)
        x = self.gat2(x, edge_index)
        x = F.log_softmax(x, dim=1)
        return x

In [15]:
class GCNNet(torch.nn.Module):
    def __init__(self):
        super(GCNNet, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.log_softmax(x, dim=1)
        
        return x


In [16]:
model = SAGENet()
print(model)

SAGENet(
  (conv1): SAGEConv(1433, 128)
  (conv2): SAGEConv(128, 7)
)


In [17]:
model = GATNet()
print(model)

GATNet(
  (gat1): GATConv(1433, 8, heads=8)
  (gat2): GATConv(64, 7, heads=1)
)


In [18]:
model = GCNNet()
print(model)

GCNNet(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
data = dataset_cora[0].to(device)
print(data)

cpu
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [20]:
import torch.optim as optim

In [21]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Test system evaluation index

In [31]:
# gpu_data = list(nvsmi.get_gpus())[0]
# start_gpu_util = gpu_data.gpu_util
# start_gpu_mem_use = gpu_data.mem_used
# total_gpu_mem = gpu_data.mem_total
disk_io_counter = psutil.disk_io_counters()
disk_total = disk_io_counter[2] + disk_io_counter[3]  # read_bytes + write_bytes
p = psutil.Process()
group_mem_rss = []
# group_gpu_util = []
# group_gpu_mem_use = []
group_disk_usage = []
group_iowait = []
t_status = True

In [32]:
def get_gpu_info():
#     gpu_data = list(nvsmi.get_gpus())[0]
    while True:
        if not t_status:
            break
#         group_gpu_util.append(gpu_data.gpu_util)
#         group_gpu_mem_use.append(gpu_data.mem_used)
        group_mem_rss.append(psutil.Process(os.getpid()).memory_info().rss)
        io_counters = p.io_counters()
        disk_usage = io_counters[2] + io_counters[3]
        group_disk_usage.append(disk_usage)  # read_bytes + write_bytes
        try:
            a = psutil.cpu_times_percent().iowait
        except:
            a = 0
        group_iowait.append(a)
        print(f"{disk_usage / 1024 / 1024} Mb/s")
        time.sleep(0.09)

# Training

In [33]:
model.train()
for epoch in range(200):
    if epoch == 1:
        t = threading.Thread(target=get_gpu_info)
        t.start()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, pred = torch.max(out[data.train_mask], dim=1)
    correct = (pred == data.y[data.train_mask]).sum().item()
    print(correct)
    acc = correct/data.train_mask.sum().item()
    print(data.train_mask.sum())

    print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(
        epoch, loss.item(), acc))
t_status = False
t.join()

140
tensor(140)
Epoch 000 train_loss: 0.0208 train_acc: 1.0000
289.13720703125 Mb/s
139
tensor(140)
Epoch 001 train_loss: 0.0268 train_acc: 0.9929
140
tensor(140)
Epoch 002 train_loss: 0.0199 train_acc: 1.0000
140
tensor(140)
Epoch 003 train_loss: 0.0235 train_acc: 1.0000
140
tensor(140)
Epoch 004 train_loss: 0.0290 train_acc: 1.0000
289.13720703125 Mb/s
140
tensor(140)
Epoch 005 train_loss: 0.0272 train_acc: 1.0000
140
tensor(140)
Epoch 006 train_loss: 0.0242 train_acc: 1.0000
140
tensor(140)
Epoch 007 train_loss: 0.0285 train_acc: 1.0000
139
tensor(140)
Epoch 008 train_loss: 0.0378 train_acc: 0.9929
140
tensor(140)
Epoch 009 train_loss: 0.0272 train_acc: 1.0000
140
tensor(140)
Epoch 010 train_loss: 0.0277 train_acc: 1.0000
289.13720703125 Mb/s
139
tensor(140)
Epoch 011 train_loss: 0.0380 train_acc: 0.9929
140
tensor(140)
Epoch 012 train_loss: 0.0163 train_acc: 1.0000
140
tensor(140)
Epoch 013 train_loss: 0.0243 train_acc: 1.0000
138
tensor(140)
Epoch 014 train_loss: 0.0409 train_acc:

140
tensor(140)
Epoch 129 train_loss: 0.0177 train_acc: 1.0000
140
tensor(140)
Epoch 130 train_loss: 0.0226 train_acc: 1.0000
140
tensor(140)
Epoch 131 train_loss: 0.0263 train_acc: 1.0000
289.13720703125 Mb/s
140
tensor(140)
Epoch 132 train_loss: 0.0176 train_acc: 1.0000
140
tensor(140)
Epoch 133 train_loss: 0.0153 train_acc: 1.0000
140
tensor(140)
Epoch 134 train_loss: 0.0174 train_acc: 1.0000
289.13720703125 Mb/s
140
tensor(140)
Epoch 135 train_loss: 0.0203 train_acc: 1.0000
140
tensor(140)
Epoch 136 train_loss: 0.0277 train_acc: 1.0000
140
tensor(140)
Epoch 137 train_loss: 0.0168 train_acc: 1.0000
289.13720703125 Mb/s
140
tensor(140)
Epoch 138 train_loss: 0.0120 train_acc: 1.0000
140
tensor(140)
Epoch 139 train_loss: 0.0294 train_acc: 1.0000
139
tensor(140)
Epoch 140 train_loss: 0.0297 train_acc: 0.9929
289.13720703125 Mb/s
140
tensor(140)
Epoch 141 train_loss: 0.0220 train_acc: 1.0000
140
tensor(140)
Epoch 142 train_loss: 0.0212 train_acc: 1.0000
140
tensor(140)
Epoch 143 train_lo

In [34]:
# print(f"GPU 显存占用: {np.mean(group_gpu_mem_use)}Mb")
# print(f"GPU 显存占用率: {np.mean(group_gpu_mem_use) * 100 / total_gpu_mem}%")
# print(f"GPU 平均使用率: {np.mean(group_gpu_util) - start_gpu_util}%")
# tmp_ = sum(np.where(np.array(group_gpu_util) - start_gpu_util > 1, True, False))
# print(f"GPU 空闲率: {(len(group_gpu_util) - tmp_) * 100 / len(group_gpu_util)}%")
print(f'内存使用：{np.mean(group_mem_rss) / 1024 / 1024 / 1024:.4f} GB')
print(f'磁盘IO使用：{np.mean(group_disk_usage) / 1024 / 1024 / 1024:.4f} GB/s')
print(f'磁盘IO使用率：{np.mean(group_disk_usage) * 100 / disk_total:.4f}%')
print(f'cpu iowait：{np.mean(group_iowait)}')

内存使用：0.4485 GB
磁盘IO使用：0.2824 GB/s
磁盘IO使用率：12.1613%
cpu iowait：0.0


In [22]:
out = model(data)
loss = criterion(out[data.test_mask], data.y[data.test_mask])
_, pred = torch.max(out[data.test_mask], dim=1)
correct = (pred == data.y[data.test_mask]).sum().item()
acc = correct/data.test_mask.sum().item()
print("test_loss: {:.4f} test_acc: {:.4f}".format(loss.item(), acc))

test_loss: 0.7891 test_acc: 0.7790


In [154]:
import os

f = os.open('file.txt', os.O_CREAT | os.O_TRUNC | os.O_DIRECT | os.O_RDWR)
s = ' ' * 1024
os.write(f, s.encode())


OSError: [Errno 22] Invalid argument